In [1]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
#헤더추가 (title,content,time)

train1 =pd.read_csv('test4.csv', engine='python' )


In [7]:
train1.head()

,인도네시아,현대,사의,산,증인,카페,바타비아,인도네시아.1,현대.1,사의.1,...,등,카페.2,명사,사진,감상,수,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,김현종,한국,인도네시아,말레이시아,논의,착수,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,인도네시아,발리,호텔,게레자,카톨릭,산토,에덴,호텔,쿠타,발리,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,인도네시아,발리,숙소,응우,라이,국제공항,노보텔,발리,응우,라이,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,프로,미스,나인,인도네시아,비행기,탈때,신발,타요,프로,미스,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,인도네시아,발리,라마야나,리조트,앤,스파,인도네시아,발리,라마야나,리조트,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
train1.shape

(49, 32)

In [10]:
train1.info

<bound method DataFrame.info of      인도네시아     현대     사의      산     증인     카페   바타비아 인도네시아.1   현대.1   사의.1  \
0      김현종     한국  인도네시아  말레이시아     논의     착수    NaN     NaN    NaN    NaN   
1    인도네시아     발리     호텔    게레자    카톨릭     산토     에덴      호텔     쿠타     발리   
2    인도네시아     발리     숙소     응우     라이   국제공항    노보텔      발리     응우     라이   
3       프로     미스     나인  인도네시아    비행기     탈때     신발      타요     프로     미스   
4    인도네시아     발리   라마야나    리조트      앤     스파  인도네시아      발리   라마야나    리조트   
5    인도네시아    영화제      개     부분  노미네이트     영화  인도네시아     영화제      개     부분   
6    인도네시아     발리     숙소      레     기안     클리      닉      인근     이비      스   
7    인도네시아     발리     숙소     아주      뷰    나투라     빌라       붓    나투라     빌라   
8       해외  인도네시아      북     루카     해상     강진    쓰나미      경보      후     해외   
9      금요일    영화계     주요     기사  인도네시아     무비      의      미래  인도네시아      편   
10      오늘      책    베트남  인도네시아     주식     투자     실전    가이드북    김재욱    베트남   
11   인도네시아     발리     숙소     기분 

In [13]:
train1.dtypes

인도네시아          object
현대             object
사의             object
산              object
증인             object
카페             object
바타비아           object
인도네시아.1        object
현대.1           object
사의.1           object
산.1            object
증인.1           object
카페.1           object
바타비아.1         object
이              object
수카르노           object
인도네시아.2        object
초대             object
대통령            object
엘리자베스          object
영국             object
여왕             object
등              object
카페.2           object
명사             object
사진             object
감상             object
수              object
Unnamed: 28    object
Unnamed: 29    object
Unnamed: 30    object
Unnamed: 31    object
dtype: object

In [21]:
#word2vec 파라미터 설명
# workers =프로세스개수
# size= 워드 벡터 특징값수(벡터의 차원을 정한다.)
# min_count =단어에 대한 최소 빈도수(적은 빈도수의 단어는 학습하지 않는다. )
# window =컨테스트 윈도우 크기
# sample=샘플링 비율 (Ex)1e-3 :0.001이 좋은 성능을 낸다

num_workers=4
num_size=300
min_word_count=1
context=10
downsampling=1e-3

In [22]:
#진행상황확인위해
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)



In [25]:
from gensim.models import word2vec
print("Training model...")
model=word2vec.Word2Vec(train1,
                       workers=num_workers,
                       size=num_size,
                       min_count=min_word_count,
                       window=context,
                       sample=downsampling)

2019-11-19 21:09:39,678 : INFO : collecting all words and their counts
2019-11-19 21:09:39,679 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2019-11-19 21:09:39,680 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-19 21:09:39,681 : INFO : collected 52 word types from a corpus of 132 raw words and 32 sentences
2019-11-19 21:09:39,682 : INFO : Loading a fresh vocabulary
2019-11-19 21:09:39,683 : INFO : effective_min_count=1 retains 52 unique words (100% of original 52, drops 0)
2019-11-19 21:09:39,684 : INFO : effective_min_count=1 leaves 132 word corpus (100% of original 132, drops 0)
2019-11-19 21:09:39,685 : INFO : deleting the raw counts dictionary of 52 items
2019-11-19 21:09:39,687 : INFO : sample=0.001 downsamples 52 most-common words
2019-11-19 21:09:39,687 : INFO : downsampling leaves estimated 35 word corpus (26.6% of prior 132)
2019-11-19 21:09:39,688 : IN

Training model...


In [27]:
#위에 껄로 학습완료

#모델 하이퍼파라미터를 설정한 내용을 모델 이름에 담는 다면 나중에 참고하기에 좋을 것이다.
#모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있다. 
model_name="300features_1minwords_10context"
model.save(model_name)

2019-11-19 21:10:13,390 : INFO : saving Word2Vec object under 300features_1minwords_10context, separately None
2019-11-19 21:10:13,392 : INFO : not storing attribute vectors_norm
2019-11-19 21:10:13,393 : INFO : not storing attribute cum_table
2019-11-19 21:10:13,398 : INFO : saved 300features_1minwords_10context


In [28]:
# 학습하기 위해서는 같은 형태의 입력값 만들어야함
# 제목마다 단어의 개수가 다르기 때문에 입력값을 하나의 형태로 만들어야함.
# 모든 단어의 벡터값에 대해 평균을 내서 제목 하나당 하나의 벡터로 만드는 방법 ????????????????????????????????????

# 하나의 제목에 대해 전체 단어의 평균값 계산

In [37]:

# 하나의 제목에 대해 전체 단어의 평균값 계산

#words:단어의 모음인 하나의 리뷰가 들어간다.????????????????????????????????????????
#model:word2vec의 모델을 넣는 곳이며, 우리가 학습한 word3vec 모델이 들어간다.
#num_features:word2vec으로 임베딩할떼 정했던 벡터의 차원 수를 뜻한다. 

words="사진조여정우아한 플라워원피스 입고"
def get_features(words,model,num_features): #여기서 words모지?
    #출력 벡터 초기화
    feature_vector=np.zeros((num_features),dtype=np.float32)
    
    num_words=0
    #어휘사전준비
    index2word_set=set(model.wv.index2word)
    
    for w in words:
        if w in index2words_set:
            num_word +=1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector=np.add(feature_vector,mode[w])

    #문장의 단어 수 만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector=np.divide(feature_vector,num_words)
    return feature_vector

In [38]:
print(feature_vector)

NameError: name 'feature_vector' is not defined

# 전체 제목에 대해 각 제목의 평균 벡터구하기

In [34]:
#title_text: 전체 제목 데이터를 입력하는 인자???????????????????
#model:word2vec 모델 입력하는 인자


def get_dataset(title_text,model,num_features):
    dataset=list()
    
    for s in title_text:
        dataset.append(get_features(s,model,num_features))
        
    titleFeatureVecs=np.stack(dataset)
        
    return titleFeatureVecs

In [35]:
print(titleFeatureVecs)

NameError: name 'titleFeatureVecs' is not defined

In [31]:
#실제 학습에 사용될 입력값
 train_data_vecs=get_dataset(train1,model,num_features)

IndentationError: unexpected indent (<ipython-input-31-713fdeaa89bc>, line 2)